In [1]:
! ls ../../data/

CrowdFlowerAnnotations.txt  Flickr8k.token.txt	       machine_translation
ExpertAnnotations.txt	    Flickr_8k.devImages.txt    readme.txt
Flicker8k_smaller	    Flickr_8k.testImages.txt
Flickr8k.lemma.token.txt    Flickr_8k.trainImages.txt


In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import random

batch_size = 16  # Batch size for training.
epochs = 35  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '../../data/machine_translation/cmn.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# for line in lines[: min(num_samples, len(lines) - 1)]:
for line in random.sample(lines, num_samples):
    input_text, target_text = line.split('\t')
    
    # We use "\t" as the "start sequence" char and "\n" as "end sequence" char
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

print("\ninput data set:")
print(input_texts[:10])
print("\ntarget data set:")
print(target_texts[:10])

2023-03-18 04:54:26.895879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 04:54:31.724660: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-18 04:54:31.724893: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-18 04:54:42.442796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Number of samples: 2000
Number of unique input tokens: 71
Number of unique output tokens: 1799
Max sequence length for inputs: 163
Max sequence length for outputs: 46

input data set:
['Dad bought me a camera.', "There's no way to win.", 'They rescued the boy from drowning.', "Tom isn't doing what he's supposed to be doing.", 'When he retired, his son took over the business.', 'Tom swims very fast.', 'Do you know the concert schedule of London Symphony Orchestra?', 'I do not know whether to accept or to refuse.', 'Come if you can.', 'Can you just please go?']

target data set:
['\t爸爸给我买了一个照相机。\n', '\t沒有辦法贏。\n', '\t他們救了這個男孩，使他免於淹死。\n', '\t湯姆沒有做他該做的事。\n', '\t他退休的时候，他儿子接手了他的生意。\n', '\t湯姆游泳游得非常快。\n', '\t你知道伦敦交响乐团的演奏会行程吗？\n', '\t我不知道是否該同意或拒絕。\n', '\t如果你能就來吧。\n', '\t能請你離開嗎？\n']


In [3]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

print("\ninput index:")
print(input_token_index)

print("\ntarget index:")
print(target_token_index)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

print("\nencoder shape:")
print(encoder_input_data.shape)
print("\ndecoder input shape:")
print(decoder_input_data.shape)
print("\ndecoder target shape:")
print(decoder_target_data.shape)
            
print("\nencoder data:")
print(encoder_input_data[-2:])
print("\ndecoder input data:")
print(decoder_input_data[-2:])
print("\ndecoder target data:")
print(decoder_target_data[-2:])


input index:
{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70}

target index:
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, ',': 5, '0': 6, '1': 7, '2': 8, '3': 9, '4': 10, '5': 11, '6': 12, '7': 13, '8': 14, '9': 15, '?': 16, 'A': 17, 'C': 18, 'D': 19, 'F': 20, 'I': 21, 'M': 22, 'T': 23, 'a': 24, 'b': 25, 'c': 26, 'e': 27, 'i': 28, 'k': 29, 'm': 30, 'o': 31, 'r': 32, 't': 33, 'w': 34, 'y': 35, '·': 36, '‘': 37, '“': 3


encoder shape:
(2000, 163, 71)

decoder input shape:
(2000, 46, 1799)

decoder target shape:
(2000, 46, 1799)

encoder data:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder input data:
[[[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder target data:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0.

# Encoder
* We discard `encoder_outputs` and only keep the states (`state_h`, `state_c`).

In [4]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

print(state_h)
print(state_c)

2023-03-18 04:55:01.379556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-18 04:55:01.383911: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-18 04:55:01.384109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7ecf76a70477): /proc/driver/nvidia/version does not exist
2023-03-18 04:55:01.395538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm/PartitionedCall:2', description="created by layer 'lstm'")
KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm/PartitionedCall:3', description="created by layer 'lstm'")


# Decoder
* Set up the decoder, using `encoder_states` as initial state.

In [5]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

print(decoder_outputs)

KerasTensor(type_spec=TensorSpec(shape=(None, None, 1799), dtype=tf.float32, name=None), name='dense/Softmax:0', description="created by layer 'dense'")


In [6]:
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 71)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 1799)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm (LSTM)                    [(None, 128),        102400      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                 

In [7]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

2023-03-18 04:55:03.531915: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 529625600 exceeds 10% of free system memory.
2023-03-18 04:55:04.808617: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 529625600 exceeds 10% of free system memory.


Epoch 1/35
100/100 [==============================] - 42s 354ms/step - loss: 1.5769 - val_loss: 1.3569
Epoch 2/35
100/100 [==============================] - 32s 317ms/step - loss: 1.4156 - val_loss: 1.3341
Epoch 3/35
100/100 [==============================] - 33s 330ms/step - loss: 1.3983 - val_loss: 1.3288
Epoch 4/35
100/100 [==============================] - 31s 309ms/step - loss: 1.3887 - val_loss: 1.3254
Epoch 5/35
100/100 [==============================] - 36s 360ms/step - loss: 1.3822 - val_loss: 1.3104
Epoch 6/35
100/100 [==============================] - 29s 292ms/step - loss: 1.3781 - val_loss: 1.3111
Epoch 7/35
100/100 [==============================] - 30s 302ms/step - loss: 1.3753 - val_loss: 1.3063
Epoch 8/35
100/100 [==============================] - 29s 288ms/step - loss: 1.3719 - val_loss: 1.3099
Epoch 9/35
100/100 [==============================] - 29s 294ms/step - loss: 1.3698 - val_loss: 1.3113
Epoch 10/35
100/100 [==============================] - 28s 285ms/step - l

# inference mode (sampling)
1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
3. Repeat with the current target token and current states

In [8]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [9]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [10]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Dad bought me a camera.
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 33ms/step
-
Input sentence: There's no way to win.
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 33ms/step
-
Input sentence: They rescued the boy from drowning.
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Tom isn't doing what he's supposed to be doing.
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 45ms/step
-
Input sentence: When he retired, his son took over the business.
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Tom swims very fast.
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Do you know the concert schedule of London Symphony Orchestra?
Decoded sentence: 我姆有了了。

1/1 [==============================] - 0s 43ms/step
-
Input sent